## TODO
 - Make a list of important dates (holidays, fast sundays, stake conferences)
 - Add a check for all future-scheduled dates that they don't have any hymns that were sung recently. This would happen if a hymn got changed after the schedule was made.

# Steps to use:
 1. Make sure the history is accurate for the past few weeks
 2. Check if there are any holidays, fast sundays, or special meetings coming up
 3. Run most of the notebook to generate suggestions
 4. Select hymns, enter on Edify
 5. Add selected hymns to history
 6. Add hymns to Gospel Library
 7. Update ward program, if needed, using Gospel Library as reference
 8. Email Ilene Shelton what the hymns are ileneshelton66@gmail.com


In [1]:
import datetime as dt

import numpy as np
import polars as pl

pl.Config().set_tbl_rows(30)
pl.Config().set_fmt_str_lengths(50)

polars.config.Config

In [2]:
def add_last_sung_col(df):
    # Load history df
    history_df = pl.read_csv('history.csv', infer_schema_length=None)
    history_df = history_df.with_columns(
        pl.col("date").str.strptime(pl.Date, "%m/%d/%Y").alias("date"),
    ).drop('name').drop_nulls()
    history_df.head()

    last_sung = history_df.group_by('id').agg(pl.max('date').alias('last_sung'))
    df = df.join(last_sung, on='id', how='left')
    return df


def get_ranking_score(df: pl.DataFrame, type_col: str, popularity_weight=0.5, noise=0.0) -> pl.Series:
    score_df = df.filter((pl.col(type_col) == 1) & (pl.col('flagged').fill_null(0) != 1))
    weeks_since_fill_value = 100
    # Add days since last sung column
    score_df = score_df.with_columns(
        ((pl.lit(dt.datetime.today()) - pl.col('last_sung')).dt.total_days()//7)
        .fill_null(weeks_since_fill_value)
        .alias('weeks_since')
    )

    midpoint = 7
    slope = 2

    def popularity_func(x): return np.round(
        1 / (1 + np.exp(-slope * (x - midpoint))), 2)
    weeks_cutoff = 8
    score_df = score_df.with_columns((pl.col('popularity') + pl.col('popularity_adjustment').fill_null(0)).alias('adj_popularity'))
    score_df = score_df.with_columns(
        pl.col('adj_popularity').map_elements(popularity_func).alias('popularity_score'),
        pl.when(pl.col('weeks_since') > weeks_cutoff)
        .then((pl.col('weeks_since') / (weeks_since_fill_value/2)).round(3))
        .otherwise(-100).alias('weeks_since_score'),
        pl.Series(np.round(np.random.uniform(-noise, noise,
                  len(score_df)), 2)).alias('noise'),
    ).with_columns(
        (pl.col('popularity_score') * popularity_weight + pl.col('weeks_since_score')
         * (1 - popularity_weight)).alias('score') + pl.col('noise')
    )

    score_df = score_df.select([
        'id', 'name', 'length', 'adj_popularity', 'weeks_since', 'popularity_score', 'weeks_since_score', 'score'
    ]).sort('score', descending=True)

    return score_df

In [3]:
df = pl.read_csv("hymns.csv")
df = add_last_sung_col(df)

## Sacrament Hymn

In [4]:
sacrament_ranking = get_ranking_score(df, 'is_sacrament', noise=0.1)
display(sacrament_ranking.head(10))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
189,"""O Thou, Before the World Began""","""3:06""",6,100,0.12,2.0,1.06
194,"""There Is a Green Hill Far Away""","""2:19""",10,21,1.0,0.42,0.81
178,"""O Lord of Hosts""","""2:12""",9,29,0.98,0.58,0.8
183,"""In Rememberance of Thy Suffering""","""3:25""",10,28,1.0,0.56,0.8
177,"""Tis Sweet to Sing the Matchless Love (HANCOCK)""","""3:08""",10,22,1.0,0.44,0.79
171,"""With Humble Heart""","""2:44""",10,19,1.0,0.38,0.78
173,"""While of These Emblems We Partake (SAUL)""","""2:53""",10,15,1.0,0.3,0.75
182,"""We'll Sing All Hail to Jesus' Name""","""2:48""",10,20,1.0,0.4,0.75
188,"""Thy Will, O Lord, Be Done""","""2:18""",10,26,1.0,0.52,0.72


## General Hymns

In [7]:
general_ranking = get_ranking_score(df, 'is_general', noise=0.15, popularity_weight=0.7)
display(general_ranking.head(30))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
21,"""Come, Listen to a Prophet's Voice""","""3:13""",7,100,0.5,2.0,1.08
157,"""Thy Spirit, Lord, Has Stirred Our Souls""","""1:39""",7,100,0.5,2.0,1.03
3,"""Now Let Us Rejoice""","""3:29""",10,23,1.0,0.46,0.978
89,"""The Lord Is My Light""","""3:39""",10,31,1.0,0.62,0.966
142,"""Sweet Hour of Prayer""","""2:58""",8,37,0.88,0.74,0.938
277,"""As I Search the Holy Scriptures""","""2:16""",8,30,0.88,0.6,0.926
308,"""Love One Another""","""1:22""",8,32,0.88,0.64,0.918
131,"""More Holiness Give Me""","""2:49""",9,22,0.98,0.44,0.908
97,"""Lead, Kindly Light""","""3:12""",9,26,0.98,0.52,0.902


# Add to History

In [6]:
raise KeyboardInterrupt("This is here to stop you from running the whole notebook.")

KeyboardInterrupt: This is here to stop you from running the whole notebook.

In [ ]:
def name_from_id(id):
    name_row = df.filter(pl.col("id") == id).select("name")
    if len(name_row) != 1:
        raise ValueError(f"Expected 1 row, got {len(name_row)}")
    return name_row["name"].item()


def format_for_csv(date, opening, sacrament, intermediate, closing):
    str_list = []
    if opening:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},opening,{opening},\"{name_from_id(opening)}\""
        )
    if sacrament:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},sacrament,{sacrament},\"{name_from_id(sacrament)}\""
        )
    if intermediate:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},intermediate,{intermediate},\"{name_from_id(intermediate)}\""
        )
    if closing:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},closing,{closing},\"{name_from_id(closing)}\""
        )
    csv_str = "\n".join(str_list)
    return csv_str

In [ ]:
date = dt.date(2024, 1, 28)
opening = 30
sacrament = 174
intermediate = 249
closing = 98

In [ ]:
csv_str = format_for_csv(date, opening, sacrament, intermediate, closing)
print("Does this look right?")
print(csv_str)

Does this look right?
01/28/2024,opening,30,"Come, Come, Ye Saints"
01/28/2024,sacrament,174,"While of These Emblems We Partake (AEOLIAN)"
01/28/2024,intermediate,249,"Called to Serve"
01/28/2024,closing,98,"I Need Thee Every Hour"


In [ ]:
# Append to history csv
with open('history.csv', 'a') as f:
    f.write(csv_str)
    f.write("\n")